In [1]:
import vk_api
import math
from PIL import Image 
from PIL import ImageDraw
import sys
import matplotlib.pyplot as plt
import matplotlib.patches as ptch

In [461]:
LOGIN = ''
PASSWORD = ''
ID = ''
FRIEND_LIMIT = 15
DISTANCE=10
IMAGE_SIZE = (1600,1600)
DEEP2 = 2
DEEP1 = 1
FIGURE_SIZE = 25

In [410]:
def log_in(user):
        vk_session = vk_api.VkApi(user.login, user.password)
        vk_session.auth()
        return vk_session.get_api()

In [323]:
def calculate_xy(parent, distance, angle):
    angle_rad = angle/360*math.pi*2
    return (parent[0]+distance*math.cos(angle_rad),parent[1]+distance*math.sin(angle_rad))

In [285]:
def calculate_centre(left_top, radius): 
    return (left_top[0]+radius/2, left_top[1]+radius/2)

In [293]:
def calculate_figure_diameter(deep):
    return FIGURE_SIZE*deep

In [294]:
def calculate_figure_radius(deep):
    return calculate_figure_diameter(deep)/2

In [390]:
def calculate_distance(deep):
    return DISTANCE*math.exp(deep*1.95)+DISTANCE

In [312]:
def draw_parent_bridge(drawer, parent, user, deep):
    centre_parent = calculate_centre(parent, calculate_figure_radius(deep+2))
    centre_user = calculate_centre(user, calculate_figure_radius(deep+1))
    drawer.line((centre_parent[0], centre_parent[1], centre_user[0], centre_user[1]), fill=128)

In [432]:
def calculate_figure_size(xy, deep):
    return (xy[0], xy[1], 
                       xy[0]+calculate_figure_diameter(deep+1), xy[1]+calculate_figure_diameter(deep+1))

In [422]:
def calculate_angeles(count):
    step = 360 / count
    angles = [counter * step for counter in range(count)]
    return angles

In [448]:
COLORS = {'root': 'red', 'opened': 'green', 'closed': 'blue', 'deactivated': 'blue'}
def calculate_figure_params(key, xy, deep):
    color = COLORS[key]
    return {'xy' : calculate_figure_size(xy, deep), 
                       'fill' : color,'outline' : None}

In [459]:
def draw_information(drawer):
    figure_params = calculate_figure_params('root', (50,50), 1)
    drawer.ellipse(xy=figure_params['xy'],
                   fill = figure_params['fill'], outline = figure_params['outline'])
    drawer.text((120,50),'root user', 128)
    figure_params = calculate_figure_params('opened', (50,120), 1)
    drawer.ellipse(xy=figure_params['xy'],
                   fill = figure_params['fill'], outline = figure_params['outline'])
    drawer.text((120,120),'opened friend', 128)
    figure_params = calculate_figure_params('closed', (50,190), 1)
    drawer.ellipse(xy=figure_params['xy'],
                   fill = figure_params['fill'], outline = figure_params['outline'])
    drawer.text((120,190),'closed friend', 128)
    figure_params = calculate_figure_params('deactivated', (50,260), 1)
    drawer.rectangle(xy=figure_params['xy'],
                   fill = figure_params['fill'], outline = figure_params['outline'])
    drawer.text((120,260),'deactivated friend', 128)

In [423]:
class UserFriends(object):
    
    def __init__(self, friends):
        self.friends = friends

    def get_friends_count(self, api):
        return len(self.friends['items'])

    def get_info(self, api, friend_id):
        user = api.users.get(user_id=friend_id)
        return user
    
    def draw(self, deep, key, api, drawer, parent):
        angles = calculate_angeles(self.get_friends_count(api))
        for friend in self.friends['items']: 
            xy = calculate_xy(parent, calculate_distance(deep+1), angles.pop(0))
            draw_parent_bridge(drawer, parent, xy, deep)
            user = User(user_id=friend['id'], first_name=friend['first_name'], last_name=friend['last_name'])
            if 'deactivated' in friend.keys():
                key = 'deactivated'
            else:
                if friend['can_access_closed']:
                    key = 'access_opened'
                else:
                    key = 'access_closed'
            user.draw(deep, key, api, drawer, parent, xy)

In [449]:
class User(object):
    
    def __init__(self, login='', password='', user_id='', first_name='', last_name=''):
        self.login = login
        self.password = password
        self.id = user_id
        self.user_friends = None
        self.first_name = first_name
        self.last_name = last_name
        self.select_draw = {'root': self.draw_root,'deactivated': self.draw_deactivated, 
                        'access_closed': self.draw_access_closed, 'access_opened':self.draw_access_opened }

    def undentify_friends(self, api, count_fr=FRIEND_LIMIT):
        self.user_friends = UserFriends(api.friends.get(user_id=self.id,order='hints', fields='nickname', count=count_fr))
        
    def get_friends(self, api, count=FRIEND_LIMIT):
        if self.user_friends is None:
            self.undentify_friends(api,count)
        return self.user_friends

    def get_friends_count(self, api):
        if self.user_friends is None:
            self.undentify_friends(api,count)
        return self.user_friends.get_friends_count(api)

    def get_info(self, api):
        return api.users.get(user_id=self.id)
    
    def draw(self, deep, key, api, drawer, parent, xy):
        if (key != "access_closed" and key != "deactivated"):
            if self.user_friends is None:
                self.undentify_friends(api)
            if deep>0:
                self.user_friends.draw(deep-1, 'parent', api, drawer, xy)
                
        self.select_draw[key](drawer, deep, xy)
        
    def draw_root(self, drawer, deep, xy):
        figure_params = calculate_figure_params('root', xy, deep)
        drawer.ellipse(xy=figure_params['xy'], 
                       fill = figure_params['fill'], outline = figure_params['outline'])
        
    def draw_deactivated(self, drawer, deep, xy):
        figure_params = calculate_figure_params('deactivated', xy, deep)
        drawer.rectangle(xy=figure_params['xy'], 
                       fill = figure_params['fill'], outline = figure_params['outline'])
    
    def draw_access_closed(self, drawer, deep, xy):
        figure_params = calculate_figure_params('closed', xy, deep)
        drawer.ellipse(xy=figure_params['xy'], 
                       fill = figure_params['fill'], outline = figure_params['outline'])
        
    def draw_access_opened(self, drawer, deep, xy):
        figure_params = calculate_figure_params('opened', xy, deep)
        drawer.ellipse(xy=figure_params['xy'], 
                       fill = figure_params['fill'], outline = figure_params['outline'])

In [460]:
def main():
    
    try:
        user=User(LOGIN, PASSWORD, ID)
        API=log_in(user)
        print('Авторизация выполнена успешно!')
    except Exception:
        print('Вы ввели неверные данные!')
        return
    
    image1 = Image.new('RGBA', (IMAGE_SIZE[0], IMAGE_SIZE[1]), (255,255,255))
    drawer1 = ImageDraw.Draw(image1)
    user.draw(DEEP1,'root', API, drawer1, (0,0), (IMAGE_SIZE[0]/2, IMAGE_SIZE[0]/2))
    draw_information(drawer1)
    image1.save('deep1.png')
    
    image2 = Image.new('RGBA', (IMAGE_SIZE[0], IMAGE_SIZE[1]), (255,255,255))
    drawer2 = ImageDraw.Draw(image2)
    user.draw(DEEP2,'root', API, drawer2, (0,0), (IMAGE_SIZE[0]/2, IMAGE_SIZE[0]/2))
    draw_information(drawer2)
    image2.save('deep2.png')   

if __name__ == '__main__':
    main()
   
 

Авторизация выполнена успешно!
